In [ ]:
import os, re, matplotlib, pandas, collections, importlib
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.stats
import pandas, re
import seaborn as sns
import scipy as sp
from typing import List, Union, Mapping, Tuple

import xlLoader
import nameUtils
importlib.reload(nameUtils)
pma_dir = '/Users/dp/pma/'

In [ ]:
def load_xl_file_with_protein_index():
    df = pandas.read_excel(f'{pma_dir}/percentCrosslinked.xlsx', sheet_name='XL')
    df = df.loc[[not pandas.isna(x) for x in df.Exp], :]
    df['Exp'] = [str(x) for x in df.Exp]
    for n, p, s in zip(df.index, df.Protein, df.Sample):
        if pandas.isna(s):
            df.loc[n, 'Sample'] = p
    print(df.tail())
    rows = df.to_dict('records')
    #print(rows)
    current_sample = ()
    _dict = {}
    for row in rows:
        sample = ';'.join((row['Exp'], row['Protein'], str(row['Sample'])))
        if current_sample == sample:
            _dict[sample][row['Label']] = row['Value']
        else:
            current_sample = sample
            _dict[sample] = row
            _dict[sample][row['Label']] = row['Value']
            del _dict[sample]['Label']
            del _dict[sample]['Value']
    _df = pandas.DataFrame(_dict).T
    print(_df.tail(2))
    return _df

_df = load_xl_file_with_protein_index()
_df.fillna(-1, inplace=True)
sns.scatterplot(y='% XL (minimal region)', x='% XL (whole lane)', data=_df, hue='Category')
plt.show()
plt.clf(); plt.close()
sns.scatterplot(y='% XL (minimal region)', x='fmol RNA (minimal region)', data=_df, hue='Category')
plt.show()
plt.clf(); plt.close()
sns.scatterplot(y='% XL (minimal region)', x='pmol protein', data=_df, hue='Category')
plt.show()
plt.clf(); plt.close()
sns.scatterplot(x='pmol protein', y='fmol RNA (minimal region)', data=_df, hue='Category')
plt.show()
plt.clf(); plt.close()
#print(_dict)

In [ ]:
importlib.reload(xlLoader)


def pal(n_colors):
    return sns.cubehelix_palette(n_colors=n_colors, start=0, gamma=3, rot=180, dark=0,)[::-1]


def get_wt_rep_mean(protein, rep, df, values_col):
    sub = df.loc[[protein==x for x in df.Protein], :]

    if len(sub.index):
        
        subrep = sub.loc[[rep==x for x in sub.Exp], :]
        subrep = subrep.loc[[x==values_col for x in subrep.Label], :]
        mean = subrep['Value'].mean()
        return mean
    else:
        print(f"No WT for {protein}, {rep}")

def get_n_replicates(protein, df, values_col):
    sub = df.loc[[protein==x for x in df.Protein], :]

    if len(sub.index):
        
        subrep = sub.loc[[x==values_col for x in sub.Label], :]
        n_reps = len(subrep.index)
        return n_reps
    return 0


def as_fraction_of_wt(
    df: pandas.DataFrame, values_label: str = '% XL (minimal region)') -> List:
    
    df.loc[:,'WT protein'] = [x.split(' ')[0] for x in df.loc[:, 'Protein']]
    df.loc[:,'WT rep mean'] = [get_wt_rep_mean(protein, rep, df, values_label) for (protein, rep) in zip(df.loc[:,'WT protein'], df.loc[:,'Exp'])]
    df.loc[:,'Value vs WT'] = [x/max([y, 10**-6]) for x,y in zip(df['Value'], df['WT rep mean'])]
    df.loc[:,'log2 Value vs WT'] = np.log2(df.loc[:, 'Value vs WT'].tolist())
    df.loc[:,'N reps'] = [get_n_replicates(protein, df, values_label) for protein in df.loc[:,'WT protein']]

####################
# Monomeric XL RNA recurrently mutated RBPs
xlLoad = xlLoader.xlLoader(f"{pma_dir}/percentCrosslinked.xlsx")
(df, recurrent) = xlLoad.load()

#print(recurrent)

proteins_ordered_by_xl_rate = recurrent['Protein'].tolist()

as_fraction_of_wt(recurrent, values_label='% XL (minimal region)')

# Plot
xl = recurrent[recurrent['Label']=='% XL (minimal region)'].copy()
xl = xl.loc[[x not in ['PCBP1 (uORF)', 'SF3B1'] for x in xl.Protein]]

print(xl['Mut/WT'].value_counts())
xl = xl.loc[[t=='Mut' for t in xl['Mut/WT']], :]
print(xl['Mut/WT'].value_counts())

xl.loc[:, 'logValue'] = np.log10(xl.loc[:,'Value']) 
sns.set_style('whitegrid')
fig = plt.figure()

print(f"{set(xl.Protein)} -> length = {len(set(xl.Protein))}")
#fig.set_size_inches(11.7, 8.27)
print(xl.head())
selected = ['PCBP1 L100P', 'PCBP1 L100Q', 'KHDRBS2 R168C',
            'A1CF E34K',  'SF3B1 K700E', 'SF3B1 E902K', 'SF3B1 R625C', 'SF3B1 R625H',
            
            'FUBP1 R429C', 'SRSF2 P95L',  'YTHDC2 E185K', 'YTHDC2 E634K',
            'RBM39 T353I', 'U2AF1 S34F',  'RARS2 R6C',  'RBFOX1 A69T',
            'HNRNPCL1 R99Q',  'DDX3X R528C',
            'NUFIP R474W' 
            'PABPC4L E312K', #'DDX50 R257Q', 'RBM11 R74C',
          
           
           ]

xlSelected = xl.copy()#loc[[x in selected for x in xl['Protein']], :]
xlSelected.index = xlSelected['Protein']
xlSelected = xlSelected.loc[selected, :]
#xlSelected = xlSelected.sort_index(by='log2 Value vs WT', ascending=False)

sns.stripplot(x='Protein', y='log2 Value vs WT', data=xlSelected,
           color='k',#'#7594AA',
                alpha=0.5,#hue='Mut/WT', #style='Exp',
            #kind='bar',
            #facecolor=(1, 1, 1, 0),
            #errcolor=".2", edgecolor='.2',
            #palette='Blues_d',#sns.cubehelix_palette(2, start=-1, rot=0.2),
           )# **get_kwargs(xl))
#plt.ylim(0,0.1)

#for protein in xl.loc[:,'Protein']:
#    n_reps = xl.loc[[x==protein for x in xl.loc[:, 'Protein']], 'N reps']
#    plt.text(protein, 2, str(n_reps.tolist()[0]))
#plt.axhspan(0.090, 0.11, facecolor='0', alpha=0.5)
#plt.text(plt.xlim()[0] + 1, 0.1, 'XL=0.1% cutoff for RBP')

plt.ylabel('log2(MUT/WT) % cross-linked molecules RNA \nper molecule of protein (minimal region)')
plt.xticks(rotation='vertical')
#sns.despine()
#plt.ylim(0,1)
plt.tight_layout()
fig.set_figheight(4)
fig.set_figwidth(6)
plt.savefig(f'{pma_dir}/dataAndScripts/clip/figs/XL_rates_recurrent_monomeric.pdf')
plt.show()
plt.clf()
plt.close()

all_wts = xl.loc[[x=='WT' for x in xl['Mut/WT']], :]
xl.loc[:,'log Value vs WT'] = np.log2(xl.loc[:, 'Value vs WT'])
xl = xl.loc[[np.isfinite(x) for x in xl['log Value vs WT']], :]
xl = xl.loc[[not pandas.isna(x) for x in xl['Mut/WT']], :]
#sns.violinplot(x='Mut/WT', y='log Value vs WT', data=xl, facecolor=None)
sns.set_style('white')
sns.distplot(xl['log Value vs WT'], hist=True, rug=True, kde=False, color='k')
sns.despine()
plt.show()
plt.clf()
plt.close()

In [ ]:
####################
# Whole lane XL RNA recurrently mutated RBPs
df = pandas.read_excel(
    #'/Users/dfporter/pma/clip/experiments/exp33 aka exp34 randos/exp33.xlsx',
    f'{pma_dir}/percentCrosslinked.xlsx',
    #sheetname='Num for fig qWB lysate 1'
    sheet_name='XL'
)
df = df.loc[[bool(pandas.isna(x)) for x in df.Discard], :]

recurrent = df.loc[[(type(x) == type('') and 'Recurrent' in x) for x in df['Category']], :]
recurrent['Mut/WT'] = [{True: 'WT', False: 'Mut'}[bool('WT' in x)] for x in recurrent.Category]
#print(recurrent)

# Put the dataframe in order
add_order_column(recurrent, values_label='% XL (whole lane)')

# Plot
xl = recurrent[recurrent['Label']=='% XL (whole lane)'].copy()
fig = plt.figure()
sns.stripplot(x='Protein', y='Value', data=xl,
           color='#7594AA', hue='Mut/WT',
            palette=pal(2), #sns.cubehelix_palette(2*len(set(randos['Protein'])), start=0, rot=-0.2),
           )# **get_kwargs(xl))
#plt.ylim(0,0.1)

plt.ylabel('% cross-linked molecules RNA \nper molecule of protein (whole lane)')
plt.xticks(rotation='vertical')
sns.despine()
#plt.ylim(0, 15)
fig.set_figwidth(6)
plt.savefig(f'{pma_dir}/dataAndScripts/clip/figs/XL_rates_recurrent_whole_lane.pdf')
plt.show()
plt.clf()

In [ ]:
clip = """
pmol protein	L5 signal (whole lane)	 L5 signal (minimal region) 	fmol RNA (whole lane)	fmol RNA (minimal region)	% XL (whole lane)	% XL (minimal region)	% lysate	Protein	Date on tube	Sample	Exp
											
											
											
											
											
											
											
0.955386499	13900	 7,730 	 1.79 	0.99717	0.3747	0.2085		EPB41L5	18-Apr	1	Exp96
0.926388942	718	 1,530 	 0.09 	0.19737	0.0200	0.0426		UBA2	21-Apr	1	Exp96
1.897855083	23300	 8,410 	 3.01 	1.08489	0.3162	0.1143		CCIN	18-Apr	1	Exp96
1.714031452				0	0.0000	0.0000			21-Apr	1	
0.932682997				0	0.0000	0.0000			21-Apr	1	
0.581156777				0	0.0000	0.0000			21-Apr	1	
3.227871227	1460	 2,180 	 0.19 	0.28122	0.0117	0.0174		ETS2	21-Apr	1	Exp96
2.870713538	133000	 68,200 	 17.16 	8.7978	1.1882	0.6111		RPL5	8-Apr	1	Exp96
5.48853132	133000	 89,300 	 17.16 	11.5197	0.6232	0.4189	0.6855	RPL5 E82K	8-Apr	1	Exp96
8.235816125	589000	 178,000 	 75.98 	22.962	1.8283	0.5561		KHDRBS2	18-Apr	1	Exp96
6.684560491	1290000	 442,000 	 166.41 	57.018	4.8580	1.6915	3.0420	KHDRBS2 R168C	18-Apr	1	Exp96
7.007224659	416000	 161,000 	 53.66 	20.769	1.5200	0.5910		KHDRBS2	21-Apr	2	Exp96
5.253283783	1020000	 403,000 	 131.58 	51.987	4.8870	1.9598	3.3159	KHDRBS2 R168C	21-Apr	2	Exp96
7.185232834	872000	 353,000 	 112.49 	45.537	3.0828	1.2595		A1CF	18-Apr	1	Exp96
8.34957344	1200000	 468,000 	 154.80 	60.372	3.6405	1.4357	1.1399	A1CF E34K	18-Apr	1	Exp96
5.423795909	679000	 266,000 	 87.59 	34.314	3.1785	1.2574		FUBP1	15-Mar	1	Exp96
4.799673291	438000	 179,000 	 56.50 	23.091	2.3270	0.9576	0.7616	FUBP1 R429C	15-Mar	1	Exp96"""
df = pandas.DataFrame([x.split('\t') for x in clip.split('\n')[1:]])
df.columns = df.iloc[0]
df = df.loc[[x!='' for x in df.Protein]]
df = df.iloc[1:]
if 'Sample' not in df.columns:
    df['Sample'] = ''
df = df[['Protein', '% XL (whole lane)', '% XL (minimal region)', 'pmol protein', 'fmol RNA (minimal region)', 'Sample', 'Exp']]
df = df.loc[[(_ is not None and bool(_) and _ != '#DIV/0!') for _ in df['% XL (minimal region)']]]
#df.index = df['Protein']

print(df)

out = []
def not_whitespace(astr: str):
    return [_ for _ in astr if _ != '']

def _mut_or_wt(protein):
    if ' ' in protein:
        return 'Recurrent, MUT'
    if protein in ['DHX21', 'NSUN2']:
        return 'RBP'
    if protein in ['BARD1', 'BRCA1', 'SMAD3']:
        return 'Putative RBP'
    return 'Recurrent, WT'

# Deal with the case of replicates.
df['Replicate'] = [':'.join(arr) for arr in zip(df.Exp, df.Protein, df.Sample)]

for rep in set(df.Replicate):
    out = df.loc[[x==rep for x in df.Replicate], :]
    exp = out['Exp'].to_list()[0]

#    out['Sample'] = out.loc[:, 'Replicate']
    del out['Sample']
    del out['Exp']
    del out['Replicate']
    protein = out.iloc[0]['Protein']
    out = out.melt()
    #print(out)
    out = [_.split('\t') for _ in out.to_csv(sep='\t').split('\n')[1:]]
    #print(out)
    out = [_ for _ in out if _!=[''] and _[1]!='Protein']
    #print(out)
    out = [';'.join([exp, protein, str(_[2]), str(_[1]), rep, '488, Typhoon', '', _mut_or_wt(protein)]) for _ in out]
    #print(out)
    print('\n'.join(out))
#pandas.DataFrame().to_string
print(df.melt())